In [6]:
# import os
# import sys

# module_path = os.path.abspath(os.path.join('../../src'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)

In [7]:
from nna.fileUtils import list_files,getLength,read_file_properties_v2

from pathlib import Path
import pandas as pd
import datetime


In [81]:
# PARAMETERS

# where to save txt file storing length info
data_folder="/home/enis/projects/nna/data/"

# path to search for audio files
# where
# ignore_folders=['/tank/data/nna/real/stinchcomb/']
ignore_folders=[]
search_path="/tank/data/nna/real/"

ignore_folders=["/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]
# search_path="/tank/data/nna/real/stinchcomb/"

# if we already have a list of files we can load them 
# files_list_path=data_folder+"stinchcomb_files_pathV1.txt"
files_list_path=data_folder+"allFields_pathV4.txt"

# if we calculated audio lengths and saved them into text file, 
# we can load them
fileswlen_path = data_folder+"allFields_wlenV4.txt"
filesWError = data_folder+"allFields_wERRORV4.txt"

# do NOT add pkl at the end
pkl_file_name="allFields_dataV4"


# this is the current info we have so we can check if we already processed a file before
current_pkl_file = "/home/enis/projects/nna/data/allFields_dataV3.pkl"

In [9]:
# Find files
# in given search path ignoring given directories
if not Path(fileswlen_path).exists():
    files_path_list=list_files(search_path,ignore_folders)
else:
    with open(files_list_path,"r") as f:
        lines=f.readlines()
        files_path_list=[line.strip() for line in lines]


In [11]:
files_path_list[0]


'/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_000000.flac'

In [32]:
from collections import Counter
# a  = Counter()
files_suffixes=[]
files_path_list_filtered=[]
for m in files_path_list:
    m=Path(m)
    mSuffix = m.suffix.lower()
    
    files_suffixes.append(mSuffix)
    if "~" in str(m):
        continue

#     if mSuffix!=".flac" and mSuffix!=".aac" and mSuffix!=".mp3":
#         print(m)
#         break
    files_path_list_filtered.append((m))
        
print(Counter(files_suffixes))
files_path_list = files_path_list_filtered[:]

Counter({'.flac': 112053, '.aac': 9101, '.mp3': 420, '.flac~': 1})


In [40]:
# Load previous data
current_file_properties_df=pd.read_pickle(str(current_pkl_file))

# remove files we already know about
currentFileSet = set(current_file_properties_df.index)
foundFileSet = set(files_path_list)
foundFileSet = foundFileSet.difference(currentFileSet)
New_files_path_list = list(foundFileSet)

In [42]:
"new",len(New_files_path_list),"previously",len(currentFileSet),"total",len(files_path_list),

('new', 76594, 'previously', 44980, 'total', 121574)

In [ ]:
# Load or calculate Audio length
import subprocess

filesWError = []

# learn length of each audio and store in a text file, 
# if file already exists, it tries to get data from there
if not Path(fileswlen_path).exists():
    length_dict={}
    for f in New_files_path_list:
#         length=float(getLength(f))
##################
        input_video = f
        ffprobe_path = '/scratch/enis/conda/envs/speechEnv/bin/ffprobe'
        cmd=[]
        cmd.extend( [ffprobe_path, '-i', '{}'.format(input_video), '-show_entries' ,'format=duration', '-v', 'quiet' ])
        result = subprocess.Popen(cmd, stdout=subprocess.PIPE,stderr=subprocess.PIPE,)
        output = result.communicate(b'\n')
        output = [i.decode('ascii') for i in output]
        if output[0]=="":
            length = -1
            print("ERROR file is too short {}".format(input_video))
            print("command run with ERROR: {}".format(cmd))
            filesWError.append(input_video)
        else:
            length = output[0].split("\n")[1].split("=")[1]
###############
        length_dict[f]=length

    length_list=list(length_dict.items())
    with open(fileswlen_path,"w") as f:
        for line in length_list:
            f.write(",".join([line[0],str(line[1])])+"\n")

with open(fileswlen_path,"r") as f:
    lines=f.readlines()
    fileswlen=[line.strip().split(",") for line in lines]

In [80]:
# print and save files with errors

print(len(filesWError))
with open(filesWError_path,"w") as f:
    for line in length_list:
        f.write(",".join([line[0],str(line[1])])+"\n")

163


In [87]:
# turn results into a dict
fileswlen=dict([(i[0],float(i[1])) for i in fileswlen])

In [127]:
# file_properties

In [128]:
# file_properties,exceptions = read_file_properties_v2(New_files_path_list,debug=0)
for f,lengthSeconds in fileswlen.items():
    if file_properties.get(Path(f)) is not None:
        file_properties[Path(f)]["durationSec"] = lengthSeconds
        file_properties[Path(f)]["timestampEnd"] = file_properties[Path(f)]["timestamp"] + datetime.timedelta(seconds=lengthSeconds)
file_properties_df=pd.DataFrame(file_properties).T


In [129]:
# exceptions

In [93]:
def str2timestamp(fileinfo_dict):
    # x=file_properties[file]
#         print(x)
    hour_min_sec=fileinfo_dict["hour_min_sec"]
    hour=int(hour_min_sec[:2])
    minute=int(hour_min_sec[2:4])
    second=int(hour_min_sec[4:6])
    year = int(fileinfo_dict["year"])

    timestamp=datetime.datetime(year, int(fileinfo_dict["month"]), int(fileinfo_dict["day"]),
                hour=hour, minute=minute, second=second, microsecond=0)
    fileinfo_dict["timestamp"]=timestamp
    return fileinfo_dict


In [130]:
# merge with previous file properties
merged_file_properties_df = pd.concat([file_properties_df,current_file_properties_df])


In [134]:
merged_file_properties_df.to_pickle(data_folder+pkl_file_name+".pkl")

In [6]:
from pathlib import Path
import pandas as pd
nnaProjectDataFolder = Path("/home/enis/projects/nna/data/")
# audio files information
file_properties_df_FilePath = nnaProjectDataFolder / "allFields_dataV3.pkl"

In [7]:
file_properties_df=pd.read_pickle(str(file_properties_df_FilePath))


In [41]:
# region_locId

In [8]:
## Load datasets
pathDatasets = "/home/enis/projects/similarSoundsApp/resources/TimeLapse_Databases/"
datasetList=[[m for m in i.glob("*") if ".ddb" in str(m) ][0] for i in (Path(pathDatasets).glob("*"))]
# datasetList[0].parent.stem
labeledImgLocations = [m.parent.stem for m in datasetList]

realFolderPath="/tank/data/nna/real/"
region_locId=[[m for m in i.glob("*") if m.is_dir() ] for i in (Path(realFolderPath).glob("*")) if i.is_dir()]

allLocations=[]
for m in region_locId:
    for n in m:
        if n.stem in labeledImgLocations:
            allLocations.append(n)


# filenamePattern="*"
# all_embeddings=[]
# for loc in allLocations:
#     # find all files
#     search_path=loc
#     all_embeddings+=list_files(str(search_path),filename=filenamePattern)

# print(len(all_embeddings))

In [25]:
labeledImgLocations.sort()
# labeledImgLocations

In [24]:
labeledImgLocations==sorted([i.stem for i in allLocations[:-9]])

True

In [26]:
allLocations[:-9]

[PosixPath('/tank/data/nna/real/prudhoe/17'),
 PosixPath('/tank/data/nna/real/prudhoe/22'),
 PosixPath('/tank/data/nna/real/prudhoe/26'),
 PosixPath('/tank/data/nna/real/prudhoe/14'),
 PosixPath('/tank/data/nna/real/prudhoe/19'),
 PosixPath('/tank/data/nna/real/prudhoe/13'),
 PosixPath('/tank/data/nna/real/prudhoe/11'),
 PosixPath('/tank/data/nna/real/prudhoe/29'),
 PosixPath('/tank/data/nna/real/prudhoe/24'),
 PosixPath('/tank/data/nna/real/prudhoe/16'),
 PosixPath('/tank/data/nna/real/prudhoe/12'),
 PosixPath('/tank/data/nna/real/prudhoe/27'),
 PosixPath('/tank/data/nna/real/prudhoe/15'),
 PosixPath('/tank/data/nna/real/anwr/36'),
 PosixPath('/tank/data/nna/real/anwr/46'),
 PosixPath('/tank/data/nna/real/anwr/31'),
 PosixPath('/tank/data/nna/real/anwr/48'),
 PosixPath('/tank/data/nna/real/anwr/42'),
 PosixPath('/tank/data/nna/real/anwr/35'),
 PosixPath('/tank/data/nna/real/anwr/38'),
 PosixPath('/tank/data/nna/real/anwr/32'),
 PosixPath('/tank/data/nna/real/anwr/45'),
 PosixPath('/ta

In [37]:
filenamePattern="*.*"
all_embeddings=[]
#only prudhoe and anwr
for loc in allLocations[:-9]:
    # find all files
    search_path=loc
    all_embeddings+=list_files(str(search_path),filename=filenamePattern)


In [38]:
len(all_embeddings)

20032

In [53]:
indexes=[str(i) for i in file_properties_df.index]
missing=[]
for m in all_embeddings:
    if m not in indexes:
        missing.append(m)
        

'/tank/data/nna/real/prudhoe/17/2019/S4A10307_20190707_054602.flac'